In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection
%ls

/content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection
 AdaBoostClassifier.joblib
 classifiers_recall_scores.joblib
 CNNClassifier.joblib
 DecisionTreeClassifier.joblib
 EDA.ipynb
 features_k_1000.joblib
 features_k_100.joblib
 features_k_10.joblib
 features_k_1200.joblib
 features_k_1500.joblib
 features_k_2000.joblib
 features_k_2500.joblib
 features_k_3000.joblib
 features_k_500.joblib
 features_k_50.joblib
 gnbClassifier.joblib
 GradientBoostingClassifier.joblib
 knnClassifier.joblib
 LSTMClassifier.joblib
'Macro Malware Detection using Machine Learning Techniques A New Approach '
 mlpClasifier.joblib
 __pycache__/
 randomForestClassifier.joblib
 recall_scores.joblib
 RobertaClassifier.joblib
 svmClassifier.joblib
 test_dataset.csv
 test_loader.joblib
 test_loader.pkl
 tfidf_1000.joblib
 tfidf_100.joblib
 tfidf_10.joblib
 tfidf_1200.joblib
 tfidf_1500.joblib
 tfidf_2000.joblib
 tfidf_2500.joblib
 tfidf_3000.joblib
 tfidf_500.joblib
 tfidf_50.joblib
 train_dataset.csv
 tra

In [ ]:
from vba_pipeline import *
import pandas as pd
from sklearn.svm import SVC
from utils import save_loader

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, recall_score
from sklearn.model_selection import GridSearchCV
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from joblib import load

from vba_pipeline import create_pipeline, evaluate_model

In [ ]:
param_grid_svm = {
    'model__C': [0.1, 1, 10, 100],
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

mapper = {
    'white' : 1,
    'mal' : 0
}

In [ ]:
train_set = pd.read_csv('train_dataset.csv', encoding='utf-16le')
val_set = pd.read_csv('validation_dataset.csv', encoding='utf-16le')
test_set = pd.read_csv('test_dataset.csv', encoding='utf-16le')

In [ ]:
x_train, y_train = train_set['vba_code'], train_set['label']
x_val, y_val = val_set['vba_code'], val_set['label']
x_test, y_test = test_set['vba_code'], test_set['label']

y_train = y_train.map(mapper)
y_val = y_val.map(mapper)
y_test = y_test.map(mapper)

In [ ]:
svm_pipeline = create_pipeline(SVC(), TfidfVectorizer())

In [ ]:
scorer = make_scorer(recall_score, average='binary')

grid_search_svm = GridSearchCV(estimator=svm_pipeline, param_grid=param_grid_svm, scoring=scorer, cv=5)

grid_search_svm

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        <vba_pipeline.PreprocessTransformer object at 0x7e055a553d90>),
                                       ('vectorizer', TfidfVectorizer()),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('model', SVC())]),
             param_grid={'model__C': [0.1, 1, 10, 100],
                         'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring=make_scorer(recall_score, response_method='predict', average=binary))

In [ ]:
grid_search_svm.fit(x_train, y_train)

print("Best parameters found: ", grid_search_svm.best_params_)
print("Best recall score: ", grid_search_svm.best_score_)

Best parameters found:  {'model__C': 0.1, 'model__kernel': 'rbf'}
Best recall score:  0.9998756218905474


In [ ]:
best_params = grid_search_svm.best_params_
svm_params = {k.replace('model__', ''): v for k, v in best_params.items()}
svm_model = SVC(**svm_params)

x_train_1000 = load('x_train_1000.joblib')

svm_model.fit(x_train_1000, y_train)

In [ ]:
save_loader('/content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection/svmClassifier.joblib' ,svm_model)